# resolución solemne 1 minería de datos

Lo primero será importar las bibliotecas de Pandas, Google Cloud, request para visualizar una api publica y Matplotlib para graficar.

In [1]:
import requests
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt


En primer lugar, visualizaré los datos provenientes de la base de datos de BigQuery. Para ser más específico, los datos consultados son los siguientes: covid19_public_forecasts.

In [2]:
client = bigquery.Client()

#contruyo una referencia para el junto de datos de covid19
dataset_ref = client.dataset("covid19_public_forecasts", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

#Enumerar todas las tablas en el conjunto de datos "covid19_public_forecasts"
tables = list(client.list_tables(dataset))

# Imprimo los nombres de todas las tablas del conjunto de datos para tener una mejor visualizacion de lo que hay
for table in tables:  
    print(table.table_id)


county_14d
county_14d_historical
county_14d_historical_
county_28d
county_28d_historical
county_28d_historical_
japan_prefecture_28d
japan_prefecture_28d_historical
japan_prefecture_28d_historical_
state_14d
state_14d_historical
state_14d_historical_
state_28d
state_28d_historical
state_28d_historical_


Del conjunto de tablas visualizadas en el código anterior, se seleccionará la tabla county_28d_historical_ para realizar una visualización de su contenido. Esta visualización será utilizada como base para llevar a cabo un análisis más detallado.

In [3]:
table = dataset_ref.table("county_28d_historical_")

# API request - fetch the table
table = client.get_table(table)
client.list_rows(table, max_results=4).to_dataframe()

,county_fips_code,county_name,state_name,forecast_date,prediction_date,new_confirmed,cumulative_confirmed,cumulative_confirmed_q0025,cumulative_confirmed_q0975,new_confirmed_7day_rolling,...,recovered,recovered_q0025,recovered_q0975,new_confirmed_ground_truth,cumulative_confirmed_ground_truth,new_deaths_ground_truth,cumulative_deaths_ground_truth,hospitalized_patients_ground_truth,recovered_documented_ground_truth,county_population
0,01001,Autauga,Alabama,2020-12-03,2020-12-12,29.854402,3154.272486,2893.0,3842.892578,29.236568,...,2783.799805,2750.033447,3439.732666,NaN,NaN,NaN,NaN,NaN,NaN,55869
1,01001,Autauga,Alabama,2020-12-03,2020-05-25,NaN,NaN,NaN,NaN,6.857143,...,NaN,NaN,NaN,9.0,168.0,0.0,3.0,NaN,NaN,55869
2,01001,Autauga,Alabama,2020-12-03,2020-12-15,30.477970,3245.082371,2893.0,4079.764893,29.855966,...,2868.187012,2827.450195,3660.731445,NaN,NaN,NaN,NaN,NaN,NaN,55869
3,01001,Autauga,Alabama,2020-12-03,2020-12-10,29.442263,3094.769638,2893.0,3667.887451,28.824234,...,2727.918457,2699.731689,3274.896240,NaN,NaN,NaN,NaN,NaN,NaN,55869


de la tabla anterior podemos observar lo siguiente:
1. **county_fips_code**: Código FIPS del condado.
2. **county_name**: Nombre del condado.
3. **state_name**: Nombre del estado.
4. **forecast_date**: Fecha de pronóstico.
5. **prediction_date**: Fecha de predicción.
6. **new_confirmed**: Nuevos casos confirmados.
7. **cumulative_confirmed**: Total acumulado de casos confirmados.
8. **cumulative_confirmed_q0025**: Percentil 0.025 del total acumulado de casos confirmados.
9. **cumulative_confirmed_q0975**: Percentil 0.975 del total acumulado de casos confirmados.
10. **new_confirmed_7day_rolling**: Nuevo número de casos confirmados en un período de 7 días.
11. **new_deaths**: Nuevas muertes.
12. **cumulative_deaths**: Total acumulado de muertes.
13. **cumulative_deaths_q0025**: Percentil 0.025 del total acumulado de muertes.
14. **cumulative_deaths_q0975**: Percentil 0.975 del total acumulado de muertes.
15. **new_deaths_7day_rolling**: Nuevo número de muertes en un período de 7 días.
16. **hospitalized_patients**: Pacientes hospitalizados.
17. **hospitalized_patients_q0025**: Percentil 0.025 de pacientes hospitalizados.
18. **hospitalized_patients_q0975**: Percentil 0.975 de pacientes hospitalizados.
19. **recovered**: Recuperados.
20. **recovered_q0025**: Percentil 0.025 de recuperados.
21. **recovered_q0975**: Percentil 0.975 de recuperados.
22. **new_confirmed_ground_truth**: Nuevos casos confirmados (verdad del terreno).
23. **cumulative_confirmed_ground_truth**: Total acumulado de casos confirmados (verdad del terreno).
24. **new_deaths_ground_truth**: Nuevas muertes (verdad del terreno).
25. **cumulative_deaths_ground_truth**: Total acumulado de muertes (verdad del terreno).
26. **hospitalized_patients_ground_truth**: Pacientes hospitalizados (verdad del terreno).
27. **recovered_documented_ground_truth**: Recuperados (verdad del terreno).
28. **county_population**: Población del condado.



Utilizando los datos obtenidos, se  genera una consulta (query) para seleccionar las columnas más relevantes de nuestro conjunto de datos. Estas columnas seleccionadas se pasarán a la biblioteca Pandas para una manipulación más eficiente y efectiva. Este proceso nos permitirá concentrarnos en la información más importante para nuestro análisis posterior.

In [6]:
query = """
        SELECT     
    county_fips_code,
    county_name,
    state_name,
    forecast_date,
    prediction_date,
    new_confirmed,
    cumulative_confirmed,
    new_confirmed_7day_rolling,
    new_deaths,
    cumulative_deaths,
    new_deaths_7day_rolling,
    county_population
        FROM `bigquery-public-data.covid19_public_forecasts.county_28d_historical_`
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 1555518269 bytes.



Como se puede observar en la parte anterior, la cantidad de memoria utilizada para visualizar esos datos es considerablemente grande. Por esta razón, se conciderará limitar la cantidad de datos o eliminar algunas columnas.

In [ ]:
query = """
        SELECT     
    county_fips_code,
    county_name,
    state_name,
    forecast_date,
    prediction_date,
    new_confirmed,
    cumulative_confirmed,
    new_confirmed_7day_rolling,
    new_deaths,
    cumulative_deaths,
    new_deaths_7day_rolling,
    county_population
        FROM `bigquery-public-data.covid19_public_forecasts.county_28d_historical_`

        LIMIT 10
        """



# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

In [10]:
query_job = client.query(query)
iformacion = query_job.to_dataframe()


         date  outcomes.death.cumulative  outcomes.death.confirmed  \
0  2021-03-07                        NaN                       NaN   
1  2021-03-06                        NaN                       NaN   
2  2021-03-05                        NaN                       NaN   
3  2021-03-04                        NaN                       NaN   
4  2021-03-03                        NaN                       NaN   

   outcomes.death.calculated.change_from_prior_day  outcomes.death.probable  \
0                                              NaN                      NaN   
1                                              NaN                      NaN   
2                                              NaN                      NaN   
3                                              NaN                      NaN   
4                                              NaN                      NaN   

   fips  outcomes.hospitalized.total  hospitalization.hospitalized.currently  \
0   NaN                 

Nombres de las columnas (titulares):
date
states
cases
testing
outcomes


Nombres de las columnas (titulares):
date
states
cases
testing
outcomes


county_14d
county_14d_historical
county_14d_historical_
county_28d
county_28d_historical
county_28d_historical_
japan_prefecture_28d
japan_prefecture_28d_historical
japan_prefecture_28d_historical_
state_14d
state_14d_historical
state_14d_historical_
state_28d
state_28d_historical
state_28d_historical_


,state_fips_code,state_name,forecast_date,prediction_date,new_confirmed,cumulative_confirmed,new_confirmed_7day_rolling,new_deaths,cumulative_deaths,new_deaths_7day_rolling,...,recovered,new_confirmed_ground_truth,cumulative_confirmed_ground_truth,new_deaths_ground_truth,cumulative_deaths_ground_truth,hospitalized_patients_ground_truth,intensive_care_patients_ground_truth,ventilator_patients_ground_truth,recovered_documented_ground_truth,state_population
0,29,Missouri,2020-08-11,2020-08-12,1000.816406,55944.191406,887.116629,3.953247,1292.452148,6.564750,...,34610.109375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6137428
1,29,Missouri,2020-08-11,2020-08-13,1417.375000,57361.566406,937.884487,10.412476,1302.864624,7.480818,...,35459.121094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6137428
2,46,South Dakota,2020-08-11,2020-07-29,NaN,NaN,149.000000,NaN,NaN,7.000000,...,NaN,149.0,8641.0,7.0,129.0,46.0,NaN,NaN,7609.0,884659
3,11,District of Columbia,2020-08-11,2020-08-03,NaN,NaN,61.333333,NaN,NaN,0.500000,...,NaN,39.0,12313.0,0.0,586.0,104.0,24.0,12.0,9893.0,702455
4,54,West Virginia,2020-08-11,2020-07-30,NaN,NaN,127.500000,NaN,NaN,2.000000,...,NaN,98.0,6375.0,1.0,112.0,102.0,40.0,19.0,4703.0,1792147


QueryJob<project=prueba1-mineria-de-datos, location=US, id=8ea2cc43-f518-46c7-9451-26bdaa8027c2>

In [32]:
asdcasdf

QueryJob<project=prueba1-mineria-de-datos, location=US, id=d66bfe3a-cd34-475b-966e-b8b6d2bc86d8>

,state_name
0,None
1,None
2,None
3,None
4,None
...,...
2767,South Dakota
2768,South Dakota
2769,South Dakota
2770,South Dakota
